In [ ]:

#  Setting display options

from tabulate import tabulate
import pandas as pd
   #
pd.set_option("display.width", 480)

print("--")


In [ ]:

from katana import remote

my_client = remote.Client()

print(my_client)


In [ ]:
NUM_PARTITIONS  = 3
   #
DB_NAME         = "my_db"
GRAPH_NAME      = "my_graph"

print("--")


In [ ]:
#  CONNECT TO GRAPH

for l_graph in my_client.get_database(name=DB_NAME).graphs_in_database():
   if (l_graph.name == GRAPH_NAME):
      my_graph=my_client.get_database(name=DB_NAME).get_graph_by_id(id=l_graph.graph_id)
         #
      break

print(my_graph)

#  Running graph.project() ..

In [5]:

#  Here we'll add stuff to the graph without a weight proprty key that analytics.sssp() will not find ..


l_query = """
   CREATE ( n: Restaurant { restaurant_code: 'PAP' } )               //  Notice single curly braces
   SET n.restaurant_name = 'Pappadeauxs Seafood Kitchen' 
   """
      #
display(l_result1 = my_graph.query(l_query))

l_query = """
   MATCH
      (n: Airport   ),
      (m: Restaurant)
   WHERE n.id               = 'SJC'
   AND   m.restaurant_code  = 'PAP'
   CREATE (n) -[r: Attraction { DRIVABLE: 'Yes' }]-> (m)
   """
      #
display(l_result2 = my_graph.query(l_query))


          0/? [?op/s]

          0/? [?op/s]

In [ ]:
#  Check the output from routine above ..

l_result = my_graph.query(
   """
   MATCH (n) 
   RETURN n
   """ )
# l_result.view()
print(tabulate(l_result, headers='keys', tablefmt='psql'))



#  Actual analytics ..

In [ ]:

#  Running analytics.sssp() should be run against a projection, a sub-graph ..

my_graph2 = my_graph.project(edge_types=["FLIES_TO"])


l_source_nodeid       = "2"                        #  id of node to start on (this is SJC)
l_weight_property     = "DISTANCE"                 #  Edge property
l_max_iterations      = 10000 

#  Generate a unique prop name so we can run this over and over without edit ..
#
l_cntr += 1
   #
l_output_propname     = "sssp_dist_" + str(l_cntr).zfill(4)

analytics.sssp(
   my_graph2,
   source               = l_source_nodeid,
   weight_property      = l_weight_property,
   max_iterations       = l_max_iterations,
   result_property_name = l_output_propname,
   )

display(print("--"))




In [ ]:
#  Check the output from routine above ..

l_result = my_graph.query(
   """
   MATCH (n) 
   RETURN n
   """ )
# l_result.view()
print(tabulate(l_result, headers='keys', tablefmt='psql'))

